In [2]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pickle
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive
import seaborn as sns
from scipy.signal import detrend
import pickle

In [3]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

Software versions
Python 3.8.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.22.0
OS Windows 10 10.0.22621 SP0
numpy 1.20.1
scipy 1.6.2
pandas 1.2.4
matplotlib 3.3.4
seaborn 0.11.1
Tue Jun 27 20:29:13 2023 Hora oficial do Brasil

In [4]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
#local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
#path2 = '/home/rnwatanabe/copdata'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

C:\\Users\\ferna\\Downloads\\IC- dados\\Dados\PDSinfo.txt
Information of 49 subjects loaded (588 rows, 29 columns).


In [5]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
ei = PDSinfo[PDSinfo['Subject']==44]
PDSinfo.memory_usage()

Index                 128
Trial                4704
Subject              4704
Vision               4704
Surface              4704
Rep                  4704
Age                  4704
AgeGroup             4704
Gender               4704
Height               4704
Mass                 4704
BMI                  4704
FootLen              4704
DominantLeg          4704
Nationality          4704
SkinColor            4704
Ystudy               4704
Footwear             4704
Illness              4704
Illness2             4704
Nmedication          4704
Medication           4704
Ortho-Prosthesis     4704
Ortho-Prosthesis2    4704
Disability           4704
Disability2          4704
Falls12m             4704
PhysicalActivity     4704
Sequence             4704
Date                 4704
dtype: int64

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
fs = 100 # Hz
import pickle 
#file = open('important', 'wb')

COPX = np.zeros((49*12, 60*fs-30))
COPZ = np.zeros((49*12, 60*fs-30))

COPXreal = np.zeros((49*12, 60*fs-30))
COPYreal = np.zeros((49*12, 60*fs-30))
COPZreal = np.zeros((49*12, 60*fs-30))

d1x = np.zeros((49*12, 60*fs-30))
d2x = np.zeros((49*12, 60*fs-30))

d1z = np.zeros((49*12, 60*fs-30))
d2z = np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))

alpha = np.zeros((49*12, 60*fs-30))


COGX = np.empty((49*12, 60*fs-30))
print(COGX.shape)
COGY = np.empty((49*12, 60*fs-30))
COGZ = np.empty((49*12, 60*fs-30))

d = np.zeros((49*12, 60*fs-30))


angulox = np.zeros((49*12, 60*fs-30))
anguloz= np.zeros((49*12, 60*fs-30))

matrizangx = np.zeros((49*12, 60*fs-30))
matrizangz = np.zeros((49*12, 60*fs-30))



subject = np.zeros((0,1))
j=0


trial = 0
for s in range (1,50):
    
    #a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    ei = PDSinfo[PDSinfo['Subject']==s].index.tolist()
    
    
    for i in range(0,len(ei)):
        a = ei[i]
        fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
        fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
        grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
        mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
        
        
        COGX[trial,:]=(mkr['COG_X'].values[30:])
        COGY[trial,:]=(mkr['COG_Y'].values[30:])
        COGZ[trial,:]=(mkr['COG_Z'].values[30:])
        
        
        mass=PDSinfo['Mass']
        if((np.isnan(d)).any) != 'True':
            d[trial,:] = np.sqrt(COGX[trial,:]**2+COGY[trial,:]**2+COGZ[trial,:]**2)
            angulox[trial,:] = np.arcsin(COGX[trial,:]/d[trial,:])
            anguloz[trial,:] = np.arcsin(COGZ[trial,:]/d[trial,:])
            sinx = np.sin(angulox[trial,:])
            sinz = np.sin(anguloz[trial,:])
            cosx = np.cos(angulox[trial,:])
            cosz = np.cos(anguloz[trial,:])
            tanx = np.tan(angulox[trial,:])
            tanz = np.tan(anguloz[trial,:])
            matrizangx[trial,:] = angulox[trial,:]
            matrizangz[trial,:] = anguloz[trial,:]


            Jb=(4/3)*(mass.values[s])*((d[trial,:])**2)
            t=grf['Time']
            dt=t[1]-t[0]


            d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
            d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

            d2x[trial,:] = np.gradient(d1x[trial,:],dt)
            d2z[trial,:] = np.gradient(d1z[trial,:],dt)

            COPX[trial,:] = d[trial,:]*sinx + Jb/(mass.values[s]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
            COPZ[trial,:] = d[trial,:]*sinz + Jb/(mass.values[s]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])




            COPXreal[trial,:] = grf['COPNET_X'].values[30:]
            COPZreal[trial,:] = grf['COPNET_Z'].values[30:]

            subject = np.vstack((subject, [s]))

            trial = trial + 1


(588, 5970)


In [7]:
MASS = (PDSinfo['Mass'])
ASISX=(mkr['R.ASIS_X'])
ASISY=(mkr['R.ASIS_Y'])
ASISZ=(mkr['R.ASIS_Z'])

GTRX = mkr['R.GTR_X']
GTRY = mkr['R.GTR_Y']
GTRZ =mkr['R.GTR_Z']

ILIACX = mkr['R.Iliac.Crestv_X']
ILIACY= mkr['R.Iliac.Crestv_Y']
ILIACZ= mkr['R.Iliac.Crestv_Z']

PSISX = mkr['R.PSIS_X']
PSISY = mkr['R.PSIS_Y']
PSISZ = mkr['R.PSIS_Z']



In [8]:
def fun(mass,x,y,z):
    #CENTRODE PRESSAO
    COPXpart = np.zeros((49*12, 60*fs-30))
    COPZpart = np.zeros((49*12, 60*fs-30))

    COPXreal = np.zeros((49*12, 60*fs-30))
    COPYreal = np.zeros((49*12, 60*fs-30))
    COPZreal = np.zeros((49*12, 60*fs-30))

    d1x = np.zeros((49*12, 60*fs-30))
    d2x = np.zeros((49*12, 60*fs-30))

    d1z = np.zeros((49*12, 60*fs-30))
    d2z = np.zeros((49*12, 60*fs-30))

    matrizangx = np.zeros((49*12, 60*fs-30))
    matrizangz = np.zeros((49*12, 60*fs-30))

    alpha = np.zeros((49*12, 60*fs-30))


    PARTX= np.empty((49*12, 60*fs-30))
    PARTY= np.empty((49*12, 60*fs-30))
    PARTZ = np.empty((49*12, 60*fs-30))

    MASS = np.empty((49*12))

    d = np.zeros((49*12, 60*fs-30))

    angulox = np.zeros((49*12, 60*fs-30))
    anguloz= np.zeros((49*12, 60*fs-30))

    matrizangx = np.zeros((49*12, 60*fs-30))
    matrizangz = np.zeros((49*12, 60*fs-30))



    subject = np.zeros((0,1))
    j=0


    trial = 0
    
    for s in range (1,50):
        for i in range(0,len(ei)):
            PARTX[trial,:]=(x.values[30:])
            PARTY[trial,:]=(y.values[30:])
            PARTZ[trial,:]=(z.values[30:])
        
        
            MASS[trial,] = (mass).values[s]
           
            d[trial,:] = np.sqrt(PARTX[trial,:]**2+PARTY[trial,:]**2+PARTZ[trial,:]**2)
            if (any(np.isnan(d[trial,:]))) != 'True':
                angulox[trial,:] = np.arcsin(PARTX[trial,:]/d[trial,:])
                anguloz[trial,:] = np.arcsin(PARTZ[trial,:]/d[trial,:])
                sinx = np.sin(angulox[trial,:])
                sinz = np.sin(anguloz[trial,:])
                cosx = np.cos(angulox[trial,:])
                cosz = np.cos(anguloz[trial,:])
                tanx = np.tan(angulox[trial,:])
                tanz = np.tan(anguloz[trial,:])
                matrizangx[trial,:] = angulox[trial,:]
                matrizangz[trial,:] = anguloz[trial,:]
            


                Jb=(4/3)*(MASS[trial,])*((d[trial,:])**2)
                t=grf['Time']
                dt=t[1]-t[0]


                d1x[trial,:] = np.gradient(matrizangx[trial,:],dt)
                d1z[trial,:] = np.gradient(matrizangz[trial,:],dt)

                d2x[trial,:] = np.gradient(d1x[trial,:],dt)
                d2z[trial,:] = np.gradient(d1z[trial,:],dt)

                COPXpart[trial,:] = d[trial,:]*sinx + Jb/(MASS[trial,]*9.8)*(sinx*d1x[trial,:]**2-cosx*d2x[trial,:]) 
                COPZpart[trial,:] = d[trial,:]*sinz + Jb/(MASS[trial,]*9.8)*(sinz*d1z[trial,:]**2-cosz*d2z[trial,:])


                COPXreal[trial,:] = grf['COPNET_X'].values[30:]
                COPZreal[trial,:] = grf['COPNET_Z'].values[30:]


                subject = np.vstack((subject, [s]))

                trial = trial + 1
    #zeros
    testex=np.where(np.isnan(COGX))
    testey=np.where(np.isnan(COGY))
    testez=np.where(np.isnan(COGZ))


    cogx=np.delete(COGX,[testex[0]],0)
    cogy=np.delete(COGY,[testey[0]],0)
    cogz=np.delete(COGZ,[testez[0]],0)

    subjectc = np.delete(subject,[testex[0]],0)

    testex = np.where(np.isnan(PARTX))
    testey = np.where(np.isnan(PARTY))
    testez = np.where(np.isnan(PARTZ))
    print(testex)

    partx=np.delete(PARTX,[testex[0]],0)
    party=np.delete(PARTY,[testey[0]],0)
    partz=np.delete(PARTZ,[testez[0]],0)
    print(np.where(np.isnan(partz)))

    subjecta = np.delete(subject,[testex[0]],0)



    testex=np.where(np.isnan(COPX))
    testez=np.where(np.isnan(COPZ))

    copx=np.delete(COPX,[testex[0]],0)
    copz=np.delete(COPZ,[testez[0]],0)
    subjectp = np.delete(subject,[testex[0]],0)
    
    #
    
    indices = np.arange(1,50).reshape(-1,1)
    np.random.shuffle(indices)
    train = indices[:40]
    test = indices[40:]


    #masstrain = mass.values[train]
    masstrain = MASS[np.in1d(subject,train),]
    print(train.shape)

    masstest = MASS[np.in1d(subject,test),]

    PARTXtrain = partx[np.in1d(subjecta,train),]
    COGXtrain = cogx[np.in1d(subjectc,train),]
    COGXtest = cogx[np.in1d(subjectc,test),]
    PARTXtest = partx[np.in1d(subjecta,test),]
    COPXtrain = copx[np.in1d(subjectp,train),]
    COPXtest = copx[np.in1d(subjectp,test),]

    PARTYtrain = party[np.in1d(subjecta,train),]
    COGYtrain = cogy[np.in1d(subjectc,train),]
    COGYtest = cogy[np.in1d(subjectc,test),]
    PARTYtest = party[np.in1d(subjecta,test),]

    PARTZtrain = partz[np.in1d(subjecta,train),]
    PARTZtest = partz[np.in1d(subjecta,test),]
    COPZtrain = copz[np.in1d(subjectp,train),]
    COPZtest = copz[np.in1d(subjectp,test),]
    COGZtrain = cogz[np.in1d(subjectc,train),]
    COGZtest = cogz[np.in1d(subjectc,test),]
    
    values = ((COPXreal,COPZreal,COPXpart,COPZpart,d1x,d1z,d2x,d2z,matrizangx,matrizangz,COGX,COGY,COGZ,PARTX,PARTY,PARTZ,angulox,anguloz,d,alpha,subject,ei,sinx,sinz,cosx,cosz,tanx,tanz,COGXtrain,COGYtrain,COGZtrain,COGXtest,COGYtest,COGZtest,PARTXtrain,PARTYtrain,PARTZtrain,PARTXtest,PARTYtest,PARTZtest,COPXtrain,COPXtest,COPZtrain,COPZtest,train,test,masstrain,masstest,subjectp,subjectc,subjecta,t))


    return  values

In [14]:
file = open('storeasis.pkl', 'wb')

         
pickle.dump((fun(MASS,ASISX,ASISY,ASISZ)), file)


file.close()
    

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(40, 1)
0       0.377840
1       0.377851
2       0.377862
3       0.377873
4       0.377882
          ...   
5995    0.378639
5996    0.378665
5997    0.378678
5998    0.378682
5999    0.378682
Name: R.ASIS_X, Length: 6000, dtype: float64


ILIAC

In [15]:
file = open('storegtr2.pkl', 'wb')

         
pickle.dump((fun(MASS,GTRX,GTRY,GTRZ)), file)


file.close()

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(40, 1)
0       0.275980
1       0.276002
2       0.276018
3       0.276023
4       0.276016
          ...   
5995    0.275096
5996    0.275105
5997    0.275084
5998    0.275038
5999    0.274979
Name: R.GTR_X, Length: 6000, dtype: float64


In [11]:
file = open('storeili2.pkl', 'wb')

         
pickle.dump((fun(MASS,ILIACX,ILIACY,ILIACZ)), file)


file.close()

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(40, 1)


In [12]:
file = open('storepsi2.pkl', 'wb')

         
pickle.dump((fun(MASS,PSISX,PSISY,PSISZ)), file)


file.close()

(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))
(40, 1)
